In [1]:
!pip install transformers
!pip install tweet-preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
from transformers import BertTokenizer, BertModel

In [3]:
from google.colab import files
import io
uploaded = files.upload()

Saving all-threads.csv to all-threads.csv


In [4]:
import pandas as pd

#For the time being we will be testing on one thread until we feel like there are enough features and until we can actually load them into our model
pheme = pd.read_csv("all-threads.csv")
#pheme.drop(columns=pheme.columns[0], axis=1, inplace=True)
pheme.head(4)

,Unnamed: 0,rid,created_at,cleaned_text,followers,retweets,favourited,label
0,0,552783667052167168,07 Jan 2015 11:07:51,france people dead shooting hq satirical weekl...,129573,486,38,1
1,1,552783745565347840,07 Jan 2015 11:08:09,ten killed shooting headquarters french satiri...,529882,127,15,1
2,2,552784168849907712,07 Jan 2015 11:09:50,breaking dead shooting headquarters french sat...,499741,105,15,1
3,3,552784526955806720,07 Jan 2015 11:11:16,reuters people shot dead headquarters french s...,1377384,412,32,1


In [5]:
print(pheme.shape)

(6416, 8)


In [6]:
import preprocessor as p
import numpy as np

false_rumour = []
false_rumour_label = []
false_rumour_followers = []
false_rumour_retweets = []
false_rumour_favourited = []

true_rumour = []
true_rumour_label = []
true_rumour_followers = []
true_rumour_retweets = []
true_rumour_favourited = []

non_rumour = []
non_rumour_label = []
non_rumour_followers = []
non_rumour_retweets = []
non_rumour_favourited = []


unverified_rumour = []
unverified_rumour_label = []
unverified_rumour_followers = []
unverified_rumour_retweets = []
unverified_rumour_favourited = []


p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION)

for i in range(pheme.shape[0]):#6425
  #print(pheme['Labels'][i])
  if pheme['label'][i] == '0':
    false_rumour.append(p.clean(pheme['cleaned_text'][i]))
    false_rumour_label.append(0)
   # false_rumour_followers.append(pheme['Followers'][i])
   # false_rumour_retweets.append(pheme['Retweets'][i])
   # false_rumour_favourited.append(pheme['Favourited'][i])
  
  elif pheme['label'][i] == '1':
    true_rumour.append(p.clean(pheme['cleaned_text'][i]))
    true_rumour_label.append(1)
   # true_rumour_followers.append(pheme['Followers'][i])
  #  true_rumour_retweets.append(pheme['Retweets'][i])
  #  true_rumour_favourited.append(pheme['Favourited'][i])

  elif pheme['label'][i] == '2':
    non_rumour.append(p.clean(pheme['cleaned_text'][i]))
    non_rumour_label.append(2)
  #  non_rumour_followers.append(pheme['Followers'][i])
  #  non_rumour_retweets.append(pheme['Retweets'][i])
  #  non_rumour_favourited.append(pheme['Favourited'][i])
  
  elif pheme['label'][i] == '3':
    unverified_rumour.append(p.clean(pheme['cleaned_text'][i]))
    unverified_rumour_label.append(3)
 #   unverified_rumour_followers.append(pheme['Followers'][i])
  #  unverified_rumour_retweets.append(pheme['Retweets'][i])
  #  unverified_rumour_favourited.append(pheme['Favourited'][i])

print(len(false_rumour)*0.8,len(true_rumour)*0.8,len(non_rumour)*0.8,len(unverified_rumour)*0.8)
print(len(false_rumour)+len(true_rumour)+len(non_rumour)+len(unverified_rumour)) #one data point is labeled None so it will be the length of the dataset - 1

508.8 852.0 3214.4 556.8000000000001
6415


In [7]:
tweets = np.concatenate((false_rumour[:510],true_rumour[:853],non_rumour[:3217],unverified_rumour[:557]))
labels = np.concatenate((false_rumour_label[:510],true_rumour_label[:853],non_rumour_label[:3217],unverified_rumour_label[:557]))
#tweets = np.concatenate((false_rumour[:0],true_rumour[:47],non_rumour[:61],unverified_rumour[:1]))
#labels = np.concatenate((false_rumour_label[:0],true_rumour_label[:47],non_rumour_label[:61],unverified_rumour_label[:1]))
#followers = np.concatenate((false_rumour_followers[:0],true_rumour_followers[:47],non_rumour_followers[:61],unverified_rumour_followers[:1]))
#retweets = np.concatenate((false_rumour_retweets[:0],true_rumour_retweets[:47],non_rumour_retweets[:61],unverified_rumour_retweets[:1]))
#favourited = np.concatenate((false_rumour_favourited[:0],true_rumour_favourited[:47],non_rumour_favourited[:61],unverified_rumour_favourited[:1])) 

#print(len(tweets),len(labels),len(followers),len(retweets),len(favourited))
#print(labels)

#features = np.array(list(zip(tweets,followers,retweets,favourited)))
#print(features)


In [8]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.2, random_state=42, shuffle=True)

X_train.shape

(4109,)

In [9]:
# Load pre-trained model tokenizer (vocabulary)
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True,)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def get_bert_embeddings(tokens_tensor, model):
    segments_ids = [1]*len(tokens_tensor)
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)

    token_vecs_sum = []

    for token in token_embeddings:

        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(sum_vec.clone().detach())


    return torch.stack(token_vecs_sum)

X_train_embeddings = []
for tweet in range(len(X_train)):
  encoded_dict = tokenizer.encode_plus(
                  X_train[tweet][0],                      
                  add_special_tokens = True, 
                  max_length = 64,  
                  truncation = True,         
                  pad_to_max_length = True,
                  return_attention_mask = True,   
                  return_tensors = 'pt',     
              ) 
  X_train_embeddings.append(get_bert_embeddings(encoded_dict['input_ids'], model))

X_test_embeddings = []
for tweet in range(len(X_test)):
  encoded_dict = tokenizer.encode_plus(
                  X_test[tweet][0],                      
                  add_special_tokens = True, 
                  truncation = True,
                  max_length = 64,           
                  pad_to_max_length = True,
                  return_attention_mask = True,   
                  return_tensors = 'pt',     
              ) 
  X_test_embeddings.append(get_bert_embeddings(encoded_dict['input_ids'], model))

X_train_embeddings = torch.stack(X_train_embeddings)
X_test_embeddings = torch.stack(X_test_embeddings)

y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [16]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class Net(nn.Module):
    def __init__(self,embedding_count, embedding_dimension,dropout):
        super(Net, self).__init__()
        self.static = True
        self.embed = nn.Embedding(embedding_count, embedding_dimension)
        self.convolutional_layers = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=4, stride=4),
            nn.BatchNorm1d(128),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.MaxPool1d(2, 2),
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=4, stride=4),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(2, 2),
            nn.Conv1d(in_channels=256, out_channels=512, kernel_size=4, stride=4), 
            nn.BatchNorm1d(512),
            nn.ReLU()
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=1536,out_features=768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Linear(in_features=768,out_features=256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(in_features=256,out_features=4),
        )

    def forward(self, x):
        x = self.convolutional_layers(x)
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = self.linear_layers(x)
        return x

In [12]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

In [ ]:
import random 
import time

embedding_count = X_train_embeddings.shape[1]
embedding_dimension = X_train_embeddings.shape[2]
dropout = 0.5
correct = 0
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model = Net(
    embedding_count=embedding_count,
    embedding_dimension=embedding_dimension,
    dropout=dropout,
)

n_epochs = 5
batch_size = 2
lr = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = torch.nn.CrossEntropyLoss()

def get_batch_data(x, y, batch_size):
    i, batch = 0, 0
    for batch, i in enumerate(range(0, len(x) - batch_size, batch_size), 1):
        x_batch = x[i : i + batch_size]
        y_batch = y[i : i + batch_size]
        yield x_batch, y_batch, batch
    
    if batch == 0:
        yield x, y, 1

train_losses, val_losses = [], []

for epoch in range(n_epochs):
    start_time = time.time()
    train_loss = 0

    model.train(True)
    for x_batch, y_batch, batch in get_batch_data(X_train_embeddings, y_train, batch_size):
        y_pred = model(x_batch)
        
        optimizer.zero_grad()
        loss = loss_fn(y_pred, y_batch.type(torch.LongTensor))
        loss.backward()
        optimizer.step()

        true_guess = torch.argmax(y_pred,1)
        correct += (true_guess==y_batch).sum()
        
        train_loss += loss.item()
       
    train_loss /= batch
    train_losses.append(train_loss)
    elapsed = time.time() - start_time
     


    print(f'Epoch: {epoch +1} Train Loss: {train_losses[-1]} , Time Elapsed: {elapsed/60}m')

print(y_pred.shape)
print(true_guess.shape)


In [ ]:
print(X_train_embeddings[0].shape)

In [ ]:
print(len(y_test))
guesses = []

begin = time.time()
model.eval() 
with torch.no_grad(): 
    y_preds = []
    batch = 0
    for x_batch, y_batch, batch in get_batch_data(X_test_embeddings, y_test, batch_size):
        y_pred = model(x_batch)
        true_guess = torch.argmax(y_pred,1)
        correct += (true_guess==y_batch).sum()
        for i in true_guess.tolist():
          guesses.append(i)
end = time.time()

In [ ]:
print(guesses)
print(len(y_test),len(guesses))

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix,precision_score,recall_score
import seaborn as sns
import matplotlib.pyplot as plt


f1 = f1_score(y_test[:-4], guesses,average = 'weighted') #not enough data for last batch so slice out from y_test
precision = precision_score(y_test[:-4], guesses,average = 'weighted') #not enough data for last batch so slice out from y_test
recall = recall_score(y_test[:-4], guesses,average = 'weighted') #not enough data for last batch so slice out from y_test

print(f1, precision, recall)

cm = confusion_matrix(y_test[:-4], guesses)

group_names = ['false_rumour','true_rumour','non_rumour','unverified_rumour']
cm_df = pd.DataFrame(cm,
                     index = group_names, 
                     columns = group_names)
#Plotting the confusion matrix
plt.figure(figsize=(5,4))
off_diag_mask = np.eye(*cm_df.shape, dtype=bool)
sns.heatmap(cm_df, annot=True, mask=off_diag_mask,fmt='')
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()